# Transformaciones

In [ ]:
import json
import pandas as pd
import re
import csv 
from textblob import TextBlob
from ast import literal_eval
from datetime import datetime

### 1- Dataset Australian_User_reviews.json
    Contiene datos de las recomendaciones y comentarios de los jugadores

In [ ]:
##Cargamos los datos desde la carpeta Datasets
##Obserbamos que el Dataset tiene columnas con datos anidados en forma de diccionarios por lo que
##deberemos tratarlo linea por linea

# Creamos esta funcion para corregir y cargar cada linea del archivo 'australian_user_reviews.json' original
def cargar_y_corregir_linea(linea):
    # reemplaza las comillas simples con dobles y corrige valores booleanos
    linea_corregida = linea.replace("'", '"').replace('True', 'true').replace('False', 'false')
    return json.loads(linea_corregida)

# lista para almacenar los datos desanidados
data_list = []

# extraemos números de la columna 'funny'
numero_regex = re.compile(r'\d+')

# lee el archivo 'australian_user_reviews.json' original, corrige y procesa cada línea
with open('./Datasets/australian_user_reviews.json', 'r', encoding='utf-8') as archivo_json:
    for linea in archivo_json:
        try:
            entrada = cargar_y_corregir_linea(linea)
            user_id = entrada['user_id']
            user_url = entrada['user_url']
            # iteramos sobre cada reseña
            for reseña in entrada['reviews']:
                # estraemos el numero de la columna 'funny'
                funny_valor = re.search(numero_regex, reseña.get('funny', ''))
                if funny_valor:
                    funny = int(funny_valor.group())
                else:
                    funny = None
                
                # elimina 'Posted' de la columna 'posted'
                posted = reseña.get('posted', '').replace('Posted ', '', 1) 

                reseña_dict = {
                    'user_id': user_id,
                    'user_url': user_url,
                    'funny': funny,
                    'posted': posted,
                    'item_id': int(reseña.get('item_id', '')),  # convierte a entero
                    'helpful': reseña.get('helpful', ''),
                    'recommend': reseña.get('recommend', ''),
                    'review': reseña.get('review', '')  # mantiene el texto original de la review
                }
                data_list.append(reseña_dict)
        except json.JSONDecodeError as e:
            None

# crea el DataFrame
df = pd.DataFrame(data_list)



In [ ]:
# Selecciona solo la columna 'review'
review_column = df[['review']]

# Guarda la columna en un archivo parquet
review_column.to_parquet('./Datasets/review_column.parquet', index=False)

In [ ]:
# Reemplaza los valores NaN por None en el DataFrame
df = df.where(df.notna(), None)
df


In [ ]:
#La columna ['posted'] presenta formato de fecha completo y algunas sin el año, 
# deduzco que el año corresponde al año 2016 en curzo en el momento de la extraccion de los datos
#por lo que realizaremos la extraccion, transformacion y asignacion del año correcto 

# Definimos una función para extraer y convertir la fecha
def extract_date(posted_str):
    if not isinstance(posted_str, str):
        return None
    
    # Elimina el punto y espacios en blanco al principio y al final
    date_str = posted_str.replace('.', '').strip()
    try:
        # Intenta convertir la cadena de fecha en un objeto de fecha utilizando datetime.strptime
        date_obj = datetime.strptime(date_str, '%B %d, %Y').date()
    except ValueError:
        try:
            # Intenta nuevamente con el formato '%B %d' si la conversión falla
            date_obj = datetime.strptime(date_str, '%B %d').date()
            # Si solo tiene mes y día y no se proporcionó el año, asumimos el año 2016
            date_obj = date_obj.replace(year=2016)
        except ValueError:
            # Si no se puede convertir a ninguna de las dos formas, devuelve None
            date_obj = None

    return date_obj

# Aplica la función extract_date a la columna 'posted'
df['posted'] = df['posted'].apply(extract_date)

# Mostrar las primeras filas para verificar
print(df)


In [ ]:
# Convertimos la columna 'posted' a datetime con manejo de errores
df['posted'] = pd.to_datetime(df['posted'], errors='coerce')

# Extraemos el año y sobrescribimos la columna 'posted' con estos valores, y luego convertimos a int
df['posted'] = df['posted'].dt.year.astype('Int64')


# Mostrar el DataFrame para verificar la conversión
print(df.info())

In [ ]:
#Eliminamos la columna 'user_url' porque no nos aporta informacion necesaria
df.drop('user_url', axis=1, inplace=True)
df

In [ ]:
#Eliminamos la columna 'helpful' porque no nos aporta informacion necesaria
df.drop('helpful', axis=1, inplace=True)
df

In [ ]:
#Eliminamos la columna 'funny' porque no nos aporta informacion necesaria
df.drop('funny', axis=1, inplace=True)
df

### 1a- Analisis de Sentimientos
    realizamos el analisis y adjuntamos la nueva columna al Dataset

In [ ]:
#Definimos una funcion que nos permita analizar el sentimiento de las reseñas junto con su 
# recomendación. Esta función evaluará si una reseña es positiva, neutral o negativa, 
# considerando también si la recomendación asociada es verdadera (positiva) o falsa (negativa). 
# Luego, aplicaremos esta función a cada reseña en nuestro DataFrame para obtener 
# un análisis de sentimiento más completo.

def get_sentiment_with_recommend(recommend, review):
    if isinstance(review, str):  # Verifica si el tipo de review es string
        if recommend:  # Verifica si la recomendación es verdadera
            analysis = TextBlob(str(review))  # Convierte a cadena y analiza el texto de la reseña
            polarity = analysis.sentiment.polarity
            if polarity > 0:
                return 2  # Positivo si polaridad > 0
            else:
                return 1  # Neutral si polaridad <= 0 y recommend true
        else:
            return 0  # No recomendado, se asume neutral o negativo
    else:
        return 1  # No hay reseña, se asume neutral

# Convertimos la columna 'review' a cadena (str)
df['review'] = df['review'].astype(str)

# Aplicamos la función a cada review
df['sentiment_analysis'] = df.apply(lambda x: get_sentiment_with_recommend(x['recommend'], x['review']), axis=1)

# Reemplazamos la columna 'review' con la nueva columna 'sentiment_analysis'
df.drop('review', axis=1, inplace=True)
# Renombramos a la columna 'sentiment_analysis' como 'review' 
df.rename(columns={'sentiment_analysis': 'review'}, inplace=True)

print(df)

In [ ]:
#Guardamos el df resultante en formato parquet 
df.to_parquet('./Datasets/user_reviews.parquet')

### 2- Dataset Australian_User_items.json
    Contiene datos de Usuarios como Items y Tiempo de Juego 

In [ ]:
##Cargamos los datos desde la carpeta Datasets
##Obserbamos que el Dataset tiene columnas con datos anidados en forma de diccionarios por lo que
##deberemos tratarlo linea por linea
## Las Lineas con Errores de formato no se cargan en el df (55010 lineas)

# Lista para almacenar los datos desanidados
data_list = []

# Función para corregir y cargar cada línea del archivo 'australian_users_items.json' original
def cargar_y_corregir_linea(linea):
    # reemplaza las comillas simples con dobles
    linea_corregida = linea.replace("'", '"')
    return json.loads(linea_corregida)

# Crea y abre un archivo 'australian_users_items.csv' para escribir los datos
with open('australian_users_items.csv', 'w', newline='', encoding='utf-8') as archivo_csv:
    writer = csv.writer(archivo_csv)
    
    # Escribimos los nombres de las columnas
    writer.writerow(['user_id', 'user_url', 'item_id', 'item_name', 'playtime_forever', 'playtime_2weeks'])
    
    # Inicializamos el contador de errores
    errores_count = 0

    # Lee el archivo 'australian_users_items.json' original, corrige y procesa cada línea
    with open('./Datasets/australian_users_items.json', 'r', encoding='utf-8') as archivo_json:
        for linea in archivo_json:
            try:
                entrada = cargar_y_corregir_linea(linea)
                user_id = entrada['user_id']
                user_url = entrada['user_url']
                # iteracion sobre cada items
                for articulo in entrada['items']:
                    writer.writerow([
                        user_id,
                        user_url,
                        articulo.get('item_id', ''),
                        articulo.get('item_name', ''),
                        articulo.get('playtime_forever', ''),
                        articulo.get('playtime_2weeks', ''),
                    ])
                    data_list.append({
                        'user_id': user_id,
                        'user_url': user_url,
                        'item_id': articulo.get('item_id', ''),
                        'item_name': articulo.get('item_name', ''),
                        'playtime_forever': articulo.get('playtime_forever', ''),
                        'playtime_2weeks': articulo.get('playtime_2weeks', ''),
                    })
            except json.JSONDecodeError as e:
                errores_count += 1
                print(f"Error al procesar la línea: {e}")

    print(f"Total de errores de JSONDecodeError detectados: {errores_count}")

# crearmos el DataFrame
df = pd.DataFrame(data_list)

In [ ]:
#Eliminamos la columna 'playtime_2weeks' porque no nos aporta informacion necesaria
df.drop('playtime_2weeks', axis=1, inplace=True)
df

In [ ]:
#Eliminamos la columna 'user_url' porque no nos aporta informacion necesaria
df.drop('user_url', axis=1, inplace=True)
df

In [ ]:
#Eliminamos la columna 'item_name' porque no nos aporta informacion necesaria
df.drop('item_name', axis=1, inplace=True)
df

In [ ]:
#Guardamos el df resultante en formato parquet 
df.to_parquet('./Datasets/user_items.parquet')

### 3- Dataset Output_Steam_games.json
    Contiene las descripciones de los juegos de la plataforma
    como desarrollador, año de lanzamiento y precio, etc

In [ ]:
##Cargamos los datos desde la carpeta Datasets

# Lista para almacenar los diccionarios convertidos
data_list = []

# Leer el archivo JSON línea por línea y convertir cada línea en un diccionario
with open('./Datasets/output_steam_games.json', 'r', encoding='utf-8') as file:
    for line in file:
        # Convertir la línea en un diccionario usando json.loads
        data_dict = json.loads(line)
        # Añadir el diccionario a la lista
        data_list.append(data_dict)

# Convertir la lista de diccionarios en un dataframe
df = pd.DataFrame(data_list)

# Verificar el contenido de la columna 'genre'
print(df['genres'].head())

# Convertir la lista de géneros en una sola línea de texto
df['genres'] = df['genres'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)


# Mostrar el dataframe resultante
print(df)

In [ ]:
df=df.rename(columns={'id': 'item_id'})

In [ ]:
# Seleccionamos las columnas que no nos aportan informacion necesaria
columnas_a_eliminar = ['title','tags', 'url', 'reviews_url', 'specs']

# Eliminamos las columnas
df = df.drop(columns=columnas_a_eliminar)

# Mostrar el DataFrame resultante
print(df)

In [ ]:
# Función para convertir los valores de 'price' a float
def convertir_price(valor):
    if valor is None:
        return 0.0  # Asigna 0.0 a los valores None
    try:
        return float(valor)
    except ValueError:
        return 0.0  # Asigna 0.0 a los valores no numéricos

# Aplicar la función a la columna 'price'
df['price'] = df['price'].apply(convertir_price)

# Mostrar el DataFrame resultante
print(df)

In [ ]:
# Convertimos la columna 'release_date' a datetime con manejo de errores
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Extraemos el año y sobrescribimos la columna 'release_date' con estos valores, y luego convertimos a int
df['release_date'] = df['release_date'].dt.year.astype('Int64')


# Mostrar el DataFrame para verificar la conversión
print(df.info())

In [ ]:
# Eliminamos filas con datos faltantes
df=df.dropna(thresh=3)
df

In [ ]:
#Guardamos el df resultante en formato parquet 
df.to_parquet('./Datasets/steam_games.parquet')

### 4- Combinaciones de Tablas para las Funciones

#### a-Dataset Developer
    Contiene nombre de los Desarrolladores, fecha de lazamiento y precio 

In [ ]:
#Seleccionamos las columnas necesarias del df Steam_games
developer = df[['publisher', 'release_date', 'price']]

#Guardamos el df resultante en formato parquet 
developer.to_parquet('./Datasets/def_developer.parquet')

#### b- Dataset Userdata
     Contiene el precio y recomendaciones de los usuarios por juego

In [ ]:
## Para cumplir con los datos requeridos por la funcion necesitamos combinar 
## Steam_games con User_reviews usando item_id como conector 

# Cargamos los archivos Parquet
df_steam_gamer = pd.read_parquet('./Datasets/steam_games.parquet')
df_user_review = pd.read_parquet('./Datasets/user_reviews.parquet')

# Verificamos los nombres de las columnas
print("Columnas en df_steam_gamer:")
print(df_steam_gamer.columns)

print("Columnas en df_user_review:")
print(df_user_review.columns)


In [ ]:
# Convertimos 'item_id' a string en ambos DataFrames
df_steam_gamer['item_id'] = df_steam_gamer['item_id'].astype(str)
df_user_review['item_id'] = df_user_review['item_id'].astype(str)


# Seleccionamos las columnas necesarias de cada DataFrame
df_steam_gamer = df_steam_gamer[['price', 'item_id']]
df_user_review = df_user_review[['user_id', 'item_id', 'recommend']]

# Realizamos la unión usando 'item_id' como clave
df_combined = pd.merge(df_steam_gamer, df_user_review, on='item_id', how='inner')

# Mostrar el DataFrame resultante
print(df_combined.info())
print(df_combined.head())

In [ ]:
# Guardar el DataFrame combinado en un nuevo archivo Parquet
df_combined.to_parquet('./Datasets/def_userdata.parquet')

#### c- Dataset UserForGenre
      Contiene el tiempo de juego de los usuarios por año

In [ ]:
## Para cumplir con los datos requeridos por la funcion necesitamos combinar 
## Steam_games con User_items usando item_id como conector 

# Cargamos los archivos Parquet
df_steam_gamer = pd.read_parquet('./Datasets/steam_games.parquet')
df_user_items = pd.read_parquet('./Datasets/user_items.parquet')

# Verificamos los nombres de las columnas
print("Columnas en df_steam_gamer:")
print(df_steam_gamer.columns)

print("Columnas en df_user_items:")
print(df_user_items.columns)

In [ ]:
# Convertir 'item_id' a string en ambos DataFrames
df_steam_gamer['item_id'] = df_steam_gamer['item_id'].astype(str)
df_user_items['item_id'] = df_user_items['item_id'].astype(str)


# Seleccionar las columnas necesarias de df_steam_gamer
df_steam_gamer_reduced = df_steam_gamer[['release_date', 'item_id', 'genres']]

# Realizar la unión usando 'item_id' como clave
df_combined = pd.merge(df_user_items, df_steam_gamer_reduced, on='item_id', how='inner')

# Comprobar el DataFrame resultante
print("Información del DataFrame combinado:")
print(df_combined.info())
print("Primeras filas del DataFrame combinado:")
print(df_combined.head())

In [ ]:
# Guardar el DataFrame combinado en un nuevo archivo Parquet
df_combined.to_parquet('./Datasets/def_userforgenre.parquet')

#### d- Dataset Best_developer_year
      Contiene las recomendaciones de lo jugadores por items, desarroladores y año

In [ ]:
## Para cumplir con los datos requeridos por la funcion necesitamos combinar 
## Steam_games con User_reviews usando item_id como conector 

# Cargamos los archivos Parquet
df_steam_gamer = pd.read_parquet('./Datasets/steam_games.parquet')
df_user_review = pd.read_parquet('./Datasets/user_reviews.parquet')

# Verificamos los nombres de las columnas
print("Columnas en df_steam_gamer:")
print(df_steam_gamer.columns)

print("Columnas en df_user_review:")
print(df_user_review.columns)

In [ ]:
# Convertimos 'item_id' a string en ambos DataFrames
df_steam_gamer['item_id'] = df_steam_gamer['item_id'].astype(str)
df_user_review['item_id'] = df_user_review['item_id'].astype(str)


# Seleccionamos las columnas necesarias de cada DataFrame
df_steam_gamer = df_steam_gamer[['item_id', 'publisher']]
df_user_review = df_user_review[['item_id', 'recommend', 'review', 'posted']]

# Realizamos la unión usando 'item_id' como clave
df_combined = pd.merge(df_user_review,df_steam_gamer, on='item_id', how='inner')

# Mostrar el DataFrame resultante
print(df_combined.info())
print(df_combined.head())

In [ ]:
# Guardar el DataFrame combinado en un nuevo archivo Parquet
df_combined.to_parquet('./Datasets/def_best_developer_year.parquet')

#### e- Dataset Developer_reviews_analysis

In [ ]:
## Para cumplir con los datos requeridos por la funcion necesitamos combinar 
## Steam_games con User_reviews usando item_id como conector 

# Cargamos los archivos Parquet
df_steam_gamer = pd.read_parquet('./Datasets/steam_games.parquet')
df_user_review = pd.read_parquet('./Datasets/user_reviews.parquet')

# Verificamos los nombres de las columnas
print("Columnas en df_steam_gamer:")
print(df_steam_gamer.columns)

print("Columnas en df_user_review:")
print(df_user_review.columns)

In [ ]:
# Convertimos 'item_id' a string en ambos DataFrames
df_steam_gamer['item_id'] = df_steam_gamer['item_id'].astype(str)
df_user_review['item_id'] = df_user_review['item_id'].astype(str)


# Seleccionamos las columnas necesarias de cada DataFrame
df_steam_gamer = df_steam_gamer[['item_id', 'publisher']]
df_user_review = df_user_review[['user_id','item_id', 'review']]

# Realizamos la unión usando 'item_id' como clave
df_combined = pd.merge(df_user_review,df_steam_gamer, on='item_id', how='inner')

# Mostrar el DataFrame resultante
print(df_combined.info())
print(df_combined.head())

In [ ]:
# Guardar el DataFrame combinado en un nuevo archivo Parquet
df_combined.to_parquet('./Datasets/def_developer_reviews_analysis.parquet')

#### f- Dataset Recomendacion_usuario para el modelo de ML
      Contiene Datos de Generos mas recomendados y jugados por los usuarios

In [25]:
## Para cumplir con los datos requeridos por la funcion necesitamos combinar 
## Steam_games con User_reviews usando item_id como conector 

# Cargamos los archivos Parquet
df_steam_gamer = pd.read_parquet('./Datasets/steam_games.parquet')
df_user_review = pd.read_parquet('./Datasets/user_reviews.parquet')

# Verificamos los nombres de las columnas
print("Columnas en df_steam_gamer:")
print(df_steam_gamer.columns)

print("Columnas en df_user_review:")
print(df_user_review.columns)

Columnas en df_steam_gamer:
Index(['publisher', 'genres', 'app_name', 'release_date', 'price',
       'early_access', 'item_id', 'developer'],
      dtype='object')
Columnas en df_user_review:
Index(['user_id', 'posted', 'item_id', 'recommend', 'review'], dtype='object')


In [26]:
# Convertimos 'item_id' a string en ambos DataFrames
df_steam_gamer['item_id'] = df_steam_gamer['item_id'].astype(str)
df_user_review['item_id'] = df_user_review['item_id'].astype(str)

# Filtramos los datos de df_user_review donde la columna 'review' sea igual a 1 o 2
df_user_review_filtrado = df_user_review[df_user_review['review'].isin([1, 2])]

# Seleccionamos las columnas necesarias de cada DataFrame
df_steam_gamer = df_steam_gamer[['item_id', 'app_name']]
df_user_review_filtrado = df_user_review_filtrado[['user_id', 'item_id', 'review']]

# Realizamos la unión usando 'item_id' como clave
df_combined = pd.merge(df_user_review_filtrado, df_steam_gamer, on='item_id', how='inner')

# Mostrar el DataFrame resultante
print(df_combined.info())
print(df_combined.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24723 entries, 0 to 24722
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   24723 non-null  object
 1   item_id   24723 non-null  object
 2   review    24723 non-null  int64 
 3   app_name  24723 non-null  object
dtypes: int64(1), object(3)
memory usage: 772.7+ KB
None
             user_id item_id  review                          app_name
0  76561198079601835     730       1  Counter-Strike: Global Offensive
1        MeaTCompany     730       1  Counter-Strike: Global Offensive
2     Rainbow-Dashie     730       2  Counter-Strike: Global Offensive
3  76561198061252210     730       2  Counter-Strike: Global Offensive
4  76561198056741844     730       1  Counter-Strike: Global Offensive


In [27]:
# Filtrar el DataFrame para seleccionar solo una parte de las filas
df_demo = df_combined.head(10000)

# Mostrar información del DataFrame resultante
print(df_demo.info())
print(df_demo.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   10000 non-null  object
 1   item_id   10000 non-null  object
 2   review    10000 non-null  int64 
 3   app_name  10000 non-null  object
dtypes: int64(1), object(3)
memory usage: 312.6+ KB
None
             user_id item_id  review                          app_name
0  76561198079601835     730       1  Counter-Strike: Global Offensive
1        MeaTCompany     730       1  Counter-Strike: Global Offensive
2     Rainbow-Dashie     730       2  Counter-Strike: Global Offensive
3  76561198061252210     730       2  Counter-Strike: Global Offensive
4  76561198056741844     730       1  Counter-Strike: Global Offensive


In [28]:
df_demo

,user_id,item_id,review,app_name
0,76561198079601835,730,1,Counter-Strike: Global Offensive
1,MeaTCompany,730,1,Counter-Strike: Global Offensive
2,Rainbow-Dashie,730,2,Counter-Strike: Global Offensive
3,76561198061252210,730,2,Counter-Strike: Global Offensive
4,76561198056741844,730,1,Counter-Strike: Global Offensive
...,...,...,...,...
9995,Delicous_Cookies,223710,1,Cry of Fear
9996,76561198094304449,223710,1,Cry of Fear
9997,TheAussieMozzie,223710,1,Cry of Fear
9998,cbfguy,223710,1,Cry of Fear


In [30]:
# Guardar el DataFrame combinado en un nuevo archivo Parquet
df_demo.to_parquet('./Datasets/def_recomendacion_usuario.parquet')